In [1]:
push!(LOAD_PATH, "../../src")
using Meteor.TensorNetwork
using Meteor.QuantumCircuit

In [2]:
function google_circuit(m::Int, n::Int, depth::Int)
    XYT = Dict(0=>sqrtX, 1=>sqrtY, 2=>T)
    Xoperated = zeros(Bool, m, n)
    Yoperated = zeros(Bool, m, n)
    Toperated = zeros(Bool, m, n)
    CZoperated = zeros(Bool, m, n)

    ControlZ = CZ
    config1 = []
    config2 = []
    config3 = []
    config4 = []
    config5 = []
    config6 = []
    config7 = []
    config8 = []
    config1sites = []
    config2sites = []
    config3sites = []
    config4sites = []
    config5sites = []
    config6sites = []
    config7sites = []
    config8sites = []
    toind(i,j) = linear_index((m, n), (i, j), false)

    # 1 configuration
    for jj in 0:2:(n-2)
        for ii in ((div(jj,2)+1)%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config1, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config1sites, (i, j))
            push!(config1sites, (i, j+1))
        end
    end

    # 2 configuration
    for jj in 0:2:(n-2)
        for ii in ((div(jj,2))%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config2, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config2sites, (i, j))
            push!(config2sites, (i, j+1))
        end
    end

    # 5 configuration
    for jj in 1:2:(n-2)
        for ii in ((div(jj,2)+1)%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config5, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config5sites, (i, j))
            push!(config5sites, (i, j+1))
        end
    end

    # 6 configuration
    for jj in 1:2:(n-2)
        for ii in (((div(jj,2)))%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config6, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config6sites, (i, j))
            push!(config6sites, (i, j+1))
        end
    end

    # 3 configuration
    for ii in 1:2:(m-2)
        for jj in ((div(ii,2)+1)%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config3, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config3sites, (i, j))
            push!(config3sites, (i+1, j))
        end
    end

    # 4 configuration
    for ii in 1:2:(m-2)
        for jj in ((div(ii,2))%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config4, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config4sites, (i, j))
            push!(config4sites, (i+1, j))
        end
    end

    # 7 configuration
    for ii in 0:2:(m-2)
        for jj in ((div(ii,2))%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config7, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config7sites, (i, j))
            push!(config7sites, (i+1, j))
        end
    end

    # 8 configuration
    for ii in 0:2:(m-2)
        for jj in ((div(ii,2)+1)%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config8, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config8sites, (i, j))
            push!(config8sites, (i+1, j))
        end
    end

    configs=[(config1, config1sites), (config2, config2sites), (config3, config3sites),
        (config4, config4sites), (config5, config5sites), (config6, config6sites),
        (config7, config7sites), (config8, config8sites)]

    function __generate_one_body_gates()
        one_body_gates = []
        tmpXoperated = zeros(Bool, m, n)
        tmpYoperated = zeros(Bool, m, n)
        for i in 1:m
            for j in 1:n
                # Place a gate at qubit q only if this qubit is
                # occupied by a CZ gate in the previous cycle
                if CZoperated[i, j]
                    # Place a T gate at qubit q if there are no single
                    # qubit gates in the previous cycles at qubit q except
                    # for the initial cycle of Hadamard gates
                    if !Toperated[i, j]
                        push!(one_body_gates, (toind(i, j), XYT[2]))
                        Toperated[i, j] = true
                    else
                        # Any gate at qubit q should be different from
                        # the gate at qubit q in the previous cycle.
                        if tmpXoperated[i, j]
                            push!(one_body_gates, (toind(i, j), XYT[1]))
                            tmpYoperated[i, j] = true
                        else
                            if tmpYoperated[i, j]
                                push!(one_body_gates, (toind(i, j), XYT[0]))
                                tmpXoperated[i, j] = true
                            else
                                r = rand(0:1)
                                push!(one_body_gates, (toind(i, j), XYT[0]))
                                if r == 0
                                    tmpXoperated[i, j] = true
                                else
                                    tmpYoperated[i, j] = true
                                end
                            end
                        end
                    end
                end
            end
        end
        Xoperated = tmpXoperated
        Yoperated = tmpYoperated
        return one_body_gates
    end


    circuit = QCircuit()
    for i in 1:m
        for j in 1:n
            push!(circuit, (toind(i, j), H))
        end
    end
    for i in 0:(depth-1)
        present_config = i % 8 + 1
        # println("present config is $present_config")
        config, configsites = configs[present_config]
        fill!(CZoperated, false)
        for item in configsites
            CZoperated[item...] = true
        end
        one_body_gates = __generate_one_body_gates()
        for item in one_body_gates
            push!(circuit, item)
        end
        for item in config
            push!(circuit, item)
        end
        # append!(circuit, one_body_gates)
        # append!(circuit, config)
    end
    for i in 1:m
        for j in 1:n
            push!(circuit, (toind(i, j), H))
        end
    end
    return circuit
end

m = 8
n = 8
depth = 12

circuit = google_circuit(m, n, depth)

QCircuit(AbstractGate[GenericOneBodyGate{Array{Float64,2}}(1, [0.7071067811865475 0.7071067811865475; 0.7071067811865475 -0.7071067811865475]), GenericOneBodyGate{Array{Float64,2}}(9, [0.7071067811865475 0.7071067811865475; 0.7071067811865475 -0.7071067811865475]), GenericOneBodyGate{Array{Float64,2}}(17, [0.7071067811865475 0.7071067811865475; 0.7071067811865475 -0.7071067811865475]), GenericOneBodyGate{Array{Float64,2}}(25, [0.7071067811865475 0.7071067811865475; 0.7071067811865475 -0.7071067811865475]), GenericOneBodyGate{Array{Float64,2}}(33, [0.7071067811865475 0.7071067811865475; 0.7071067811865475 -0.7071067811865475]), GenericOneBodyGate{Array{Float64,2}}(41, [0.7071067811865475 0.7071067811865475; 0.7071067811865475 -0.7071067811865475]), GenericOneBodyGate{Array{Float64,2}}(49, [0.7071067811865475 0.7071067811865475; 0.7071067811865475 -0.7071067811865475]), GenericOneBodyGate{Array{Float64,2}}(57, [0.7071067811865475 0.7071067811865475; 0.7071067811865475 -0.7071067811865475

In [3]:
# gate fusion
println("total number of gates $(length(circuit)).")
pcircuit = fuse_gate(circuit)
println("number of gates after gate fusion $(length(pcircuit)).")

total number of gates 632.
number of gates after gate fusion 168.


In [4]:
# generate some random basis
N = 5
L = m * n
println("generate $N random basis...")
basis = [rand(0:1, L) for i in 1:5]

generate 5 random basis...


5-element Array{Array{Int64,1},1}:
 [0, 0, 0, 1, 0, 1, 1, 1, 1, 1  …  0, 1, 1, 0, 1, 1, 1, 1, 0, 1]
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0  …  0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
 [1, 1, 1, 1, 1, 0, 1, 0, 1, 1  …  0, 0, 1, 1, 0, 1, 0, 1, 0, 1]
 [1, 1, 0, 0, 1, 0, 1, 1, 1, 1  …  1, 0, 0, 1, 1, 1, 1, 0, 0, 0]
 [0, 1, 1, 1, 0, 0, 0, 1, 0, 0  …  0, 0, 1, 0, 1, 1, 1, 1, 0, 0]

In [5]:
# matrix product states simulation
amps = mps_amplitudes(pcircuit, basis, scaling=sqrt(2))

5-element Array{Complex{Float64},1}:
  0.20436480655664988 + 0.8030762857956077im
  -0.8035713927182326 + 0.6720364739426341im
 -0.46340543749649343 - 0.3547672023222247im
   0.7410190109241599 + 0.8799053544345506im
   0.7887720862565859 - 0.23252721130492077im

In [6]:
connectivities = square_connectivity(m, n)

112-element Array{Tuple{Int64,Int64},1}:
 (1, 2)
 (2, 3)
 (3, 4)
 (4, 5)
 (5, 6)
 (6, 7)
 (7, 8)
 (9, 10)
 (10, 11)
 (11, 12)
 (12, 13)
 (13, 14)
 (14, 15)
 ⋮
 (45, 53)
 (46, 54)
 (47, 55)
 (48, 56)
 (49, 57)
 (50, 58)
 (51, 59)
 (52, 60)
 (53, 61)
 (54, 62)
 (55, 63)
 (56, 64)

In [7]:
state = GraphState(connectivities)
apply!(pcircuit, state)
amps = amplitudes(state, basis, scaling=sqrt(2))

5-element Array{Complex{Float64},1}:
 0.20436480655663108 + 0.8030762857955595im
 -0.8035713927181827 + 0.6720364739426232im
 -0.4634054374964728 - 0.3547672023221968im
  0.7410190109241415 + 0.8799053544345021im
  0.7887720862565647 - 0.2325272113049424im

In [8]:
depth = 16
circuit = google_circuit(m, n, depth)
state = GraphState(connectivities)
apply!(circuit, state)
amps = amplitudes(state, basis, scaling=sqrt(2), cut=[(25, 33), (26, 34), (27, 35), (28, 36), (29, 37)])

5-element Array{Complex{Float64},1}:
 -0.48789050435134124 - 0.46627691790485976im
  -0.7030590318055637 - 0.5036015010984858im
  -0.8362485520894687 - 0.019588729140718512im
    1.021664075928392 - 0.2541595275158598im
  -1.0184801660854261 - 0.3303595979218251im

In [9]:
# specify a contraction path
using Meteor.QuantumCircuit: linear_index
rowmajor = false
contract_path = Int[]
shape = (m, n)
for i in 1:8
    for j in 1:4
        push!(contract_path, linear_index(shape, (i, j), rowmajor))
    end
end
for i in 8:-1:1
    for j in 5:8
        push!(contract_path, linear_index(shape, (i, j), rowmajor))
    end
end

println("contraction path $contract_path")

contraction path [1, 9, 17, 25, 2, 10, 18, 26, 3, 11, 19, 27, 4, 12, 20, 28, 5, 13, 21, 29, 6, 14, 22, 30, 7, 15, 23, 31, 8, 16, 24, 32, 40, 48, 56, 64, 39, 47, 55, 63, 38, 46, 54, 62, 37, 45, 53, 61, 36, 44, 52, 60, 35, 43, 51, 59, 34, 42, 50, 58, 33, 41, 49, 57]


In [10]:
amps = amplitudes(state, basis, scaling=sqrt(2), cut=[(25, 33), (26, 34), (27, 35), (28, 36), (29, 37)], path=contract_path)

5-element Array{Complex{Float64},1}:
 -0.4878905043513411 - 0.46627691790485726im
 -0.7030590318055655 - 0.5036015010984863im
 -0.8362485520894685 - 0.01958872914071723im
   1.021664075928392 - 0.2541595275158609im
 -1.0184801660854264 - 0.3303595979218258im